# SetUp

In [1]:
import sys

from pathlib import Path
sys.path.append('..')
from src.constants import INDIVIDUAL_DATA_OUT_PATH, HOGAR_DATA_OUT_PATH, AGLOMERADO_STR
from src.extraer_escribir_columnas import extraer
from collections import defaultdict
from src.porcentaje_techumbres import analizar_material_precario
from src.porcentaje_techumbres import  obtener_ultimo_trimestre 
from src.obtener_fechas import get_quarters
from src.ranking_cinco import *

HEADER = '-'*90
SEPARADOR = '-'*70

# Se extrae solo la información necesaria

In [2]:
columnas_individuos = ['PONDERA', 'ESTADO', 'ANO4', 'TRIMESTRE', 'CH06', 'CH09', 'AGLOMERADO', 'CAT_INAC', 'PONDERA',
                    'CODUSU', 'NRO_HOGAR', 'NIVEL_ED_str', 'ANO4', 'TRIMESTRE', 'CH15', 'CH06']

columnas_hogares = ['IV1','REGION','PONDERA','CODUSU', 'NRO_HOGAR', 'MATERIAL_TECHUMBRE',
                    'CONDICION_DE_HABITABILIDAD', 'AGLOMERADO', 'ANO4', 'TRIMESTRE', 'II7', 'IV8', 'IX_TOT']

datos_individuos = extraer(INDIVIDUAL_DATA_OUT_PATH, columnas_individuos)

datos_hogares = extraer(HOGAR_DATA_OUT_PATH, columnas_hogares)

# Ejercicio 1 parte B
A partir de la información de cada año contenida en el dataset se debe informar, año
tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y
escribir.

Importante: tomar la información del último trimestre de cada año.

In [3]:
# Declarar función contadora
def contar_analfabetos(contador_analfabetos, ANO4, TRIMESTRE, PONDERA, CH06, CH09):
    """
    Actualiza los contadores de personas mayores a 6 años incapaces de leer y escribir

    Params:
        contador_analfabetos (defaultdict): Diccionario donde para cada año hay un valor "trim" y los contadores "target" y "total".
        ANO4 (str): Año de la encuesta.
        TRIMESTRE (int): Trimestre de la encuesta.
        PONDERA (int): Ponderación de los encuestados que se suma a los contadores.
        CH06 (int): Edad del encuestado.
        CH09 (str): Numero que indica si el individuo sabe leer y escribir.
    """
    if TRIMESTRE > contador_analfabetos[ANO4]['trim']:
        contador_analfabetos[ANO4] = {'trim': TRIMESTRE, 'target': 0, 'total': 0}
    if contador_analfabetos[ANO4]['trim'] == TRIMESTRE:
        # Sumo a los totales del año
        contador_analfabetos[ANO4]['total'] += PONDERA

        # Sumo los que cumplen la condición        
        if CH06 > 6 and CH09 == '2':
            contador_analfabetos[ANO4]['target'] += PONDERA

# Declarar función que imprime
def imprimir_analfabetos(contador_analfabetos):
    """
    Calcula e imprime para el ultimo trimestre de cada año en los dataset el 
    porcentaje de personas personas mayores a 6 años incapaces de leer y escribir

    Params:
        contador_analfabetos (defaultdict): Diccionario donde para cada año hay un valor "trim" y los contadores "target" y "total".
    """

    if len(contador_analfabetos) != 0: 
        print(HEADER)

    for key in sorted(contador_analfabetos.keys(),key=int):
        total = contador_analfabetos[key]['total']
        target = contador_analfabetos[key]['target']
        perc_incapable = target / total * 100
        perc_capable = 100 - perc_incapable
        
        print(f"Año: {key}, trimestre: {contador_analfabetos[key]['trim']}")
        print(f"El porcentaje de personas mayores a 6 años capaces de leer y escribir en {key} fue {perc_capable:.2f}%")
        print(f"El porcentaje de personas mayores a 6 años incapaces de leer y escribir en {key} fue {perc_incapable:.2f}%")
        print(HEADER)

# Ejercicio 2 - Parte B
A partir de un año y trimestre elegido por el usuario informar el porcentaje de
personas no nacidas en Argentina que hayan cursado un nivel universitario o
superior

In [4]:
# Declarar función contadora
def contar_universitarios_no_argentinos(contador, anio_usuario, trimestre_usuario, ANO4, TRIMESTRE, PONDERA, CH15, NIVEL_ED_str):
    """
    Actualiza los contadores de personas no nacidas en Argentina y aquellas que hayan cursado 
    un nivel universitario o superior, siempre que se cumpla la condición del año y trimestre 
    ingresados por el usuario.

    Params:
        contador (dict): Diccionario con los contadores "total_no_nacidos_arg" y "universitarios_no_nacidos_arg".
        anio_usuario (str): Año elegido por el usuario para contar.
        trimestre_usuario (str): Trimestre elegido por el usuario para contar.
        ANO4 (str): Año de la encuesta.
        TRIMESTRE (str): Trimestre de la encuesta.
        PONDERA (int): Ponderación de los encuestados que se suma a los contadores.
        CH15 (str): Indicador del país de nacimiento del individuo; '4' o '5' determinan que no nació en Argentina.
        NIVEL_ED_str (str): Numero que representa el nivel educativo alcanzado.
    """
    # Filtro por año y trimestre
    if ANO4 == anio_usuario and TRIMESTRE == trimestre_usuario:
        
        # Evalua si el individuo NO nacio en Argentina (CH15 == '4' o CH15 == '5')
        if CH15 in ('4', '5'):
            contador["total_no_nacidos_arg"] += PONDERA # incremento el contador de no nacidos en Arg.
            
            # Filtro por nivel universitario (NIVEL_ED_str == 'Superior universitario incompleto' o 'Superior universitario completo')
            if NIVEL_ED_str == "Superior o universitario":
                contador["universitarios_no_nacidos_arg"] += PONDERA  # Incremento contador en no nacidos en Arg que hayan estudiado en ciclo universitario


# Declarar función que imprime
def imprimir_universitarios_no_argentinos(contador):
    """
    Calcula e imprime el porcentaje de personas no nacidas en Argentina que han cursado 
    un nivel universitario o superior, utilizando los contadores acumulados.

    Params:
        contador (dict): Diccionario con los contadores "total_no_nacidos_arg" y "universitarios_no_nacidos_arg".
    """
    # Calculo el porcentaje
    if contador["total_no_nacidos_arg"] > 0:
        porcentaje = (contador["universitarios_no_nacidos_arg"] / contador["total_no_nacidos_arg"]) * 100
    else:
        porcentaje = 0

    # Imprimo el resultado  
    print(f"El porcentaje de personas no nacidas en Argentina con nivel universitario o superior en el año {anio_usuario}, trimestre {trimestre_usuario}: {porcentaje:.2f}%")

 # Punto 3 Parte B

A partir de la informacion contenida en el dataset, informar el año donde hubo menor desocupacion.
	
Datos : ESTADO , PONDERA , ANO4 

In [5]:
# Declarar función contadora
def contar_desocupacion(anos_desocupacion, ESTADO, ANO4, PONDERA):
    """
    Actualiza los contadores de individios desocupados en cada año

    Params:
        anos_desocupacion (dict): Diccionario que cuenta para cada año los individos desocupados.
        ESTADO (str): Cadena que representa la condición de actividad del encuestado.
        ANO4 (str): Año de la encuesta.
        PONDERA (int): Ponderación de los encuestados que se suma a los contadores.
    """
    if ESTADO == "2":
        # Si el estado es 2, es desocupado, obtenemos el año y la ponderación
        if ANO4 in anos_desocupacion:
            anos_desocupacion[ANO4] += PONDERA
        else:
            anos_desocupacion[ANO4] = PONDERA 

# Declarar función que imprime
def imprimir_desocupacion(anos_desocupacion):
    """
    imprime para cada año su cantidad de desocupados y luego el año con menor desocupacion y su cantidad de desocupados, utilizando los contadores acumulados.

    Params:
        anos_desocupacion (dict): Diccionario que cuenta para cada año los individos desocupados.
    """
    print(HEADER)
    # Muestro la info de los años
    for ano, ponderacion in anos_desocupacion.items():
        print(f"Año: {ano}, Desocupados: {ponderacion}")

    # informar el año con menor desocupacion, usando una funcion lamba comparando los valores
    ano_menor_desocupacion = min(anos_desocupacion.items(), key=lambda x: x[1])
    print(f"\nEl año con menor desocupación es {ano_menor_desocupacion[0]} con una total de {ano_menor_desocupacion[1]} de desocupados.")
    print(HEADER)

# Obtengo la informacion del trimestre mas actual

In [6]:
try: 
    trims_hogares = get_quarters('hogar')
    trimestre_mas_actual = max(trims_hogares) if trims_hogares else (0,0)
except FileNotFoundError:
    print(f"No existe la carpeta 'hogares'.")
except PermissionError:
    print(f"No tienes permiso para acceder a la carpeta 'hogares'.")
print (trimestre_mas_actual)

(2024, 2)


# Ejercicio 4 Parte B

Ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más
ocupantes con estudios universitarios o superiores finalizados. Información obtenida
a partir del par de archivos más recientes

In [7]:
ranking = guardar_ranking_aglomerados (calcular_datos_hogares_por_aglomerado ())

/Users/bautistaameijeiras/Desktop/tp python viejo/code/notebook/../src/ranking_cinco.py:10: DtypeWarning: Columns (10,13,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hogares = pd.read_csv(HOGAR_DATA_OUT_PATH , delimiter=';')


Intentando guardar en: /Users/bautistaameijeiras/Desktop/tp python viejo/code/data_out/ranking_aglomerados.csv
CSV guardado en: /Users/bautistaameijeiras/Desktop/tp python viejo/code/data_out/ranking_aglomerados.csv


/Users/bautistaameijeiras/Desktop/tp python viejo/code/notebook/../src/ranking_cinco.py:11: DtypeWarning: Columns (102,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df_individuos = pd.read_csv(INDIVIDUAL_DATA_OUT_PATH, delimiter=';')


# Ejercicio 5 Parte B

Informar para cada aglomerado el porcentaje de viviendas ocupadas por sus 
propietarios. 

In [8]:
# Declarar función contadora
def contar_viviendas_propietarias(aglomerados,AGLOMERADO, II7, PONDERA): 
	"""
    Actualiza los contadores de viviendas ocupadas por sus propietarios. 

    Params:
        aglomerados (dict): Diccionario que cuenta para cada aglomerado las viviendas ocupadas por sus propietarios.
        AGLOMERADO (str): Cadena que representa el código del aglomerado donde se ubica el hogar encuestado.
        II7 (str): Cadena que representa el régimen de tenencia del hogar.
        PONDERA (int): Ponderación de los hogares encuestados que se suma a los contadores.
    """ 
    # si no existe el aglomerado en el nuevo diccionario lo agrego 
	if not AGLOMERADO in aglomerados:
        # total: se refiere al total de los hogares
        # propietarios: si es propietario del hogar
		aglomerados[AGLOMERADO] = {'total': 0, 'propietarios': 0}

	# agrego la cantidad de hogares de la muestra al aglomerado
	aglomerados[AGLOMERADO]['total'] += PONDERA

	# si es propietario, sumo su ponderación al contador de propietarios
	if II7 in ['1', '2']:
		aglomerados[AGLOMERADO]['propietarios'] += PONDERA


# Declarar función que imprime
def imprimir_viviendas_propietarias(aglomerados):
	"""
    imprime para cada aglomerado su porcentaje de viviendas ocupadas por sus propietarios, utilizando los contadores acumulados.

    Params:
        aglomerados (dict): Diccionario que cuenta para cada aglomerado las viviendas ocupadas por sus propietarios.
    """
	print(HEADER)
	print("Porcentaje de viviendas ocupadas por sus propietarios por cada aglomerado: \n")
	for aglomerado, datoTotal in aglomerados.items():
		tot = (datoTotal['propietarios'] / datoTotal['total']) * 100
		# busco el nombre del aglomerado, si no se encuentra devuelvo el codigo 
		nombre = AGLOMERADO_STR.get(int(aglomerado), f" aglomerado {aglomerado}")
		print(f"{nombre}: {tot:.2f} %")
	print(HEADER)


# Ejercicio 6 Parte B

Informar el aglomerado con mayor cantidad de viviendas con más de dos ocupantes 
y sin baño. Informar también la cantidad de ellas.

In [9]:
# Declarar función contadora
def contar_aglomerados_ocupados_sin_banio(aglomerados,AGLOMERADO,IV8,IX_TOT,PONDERA):
    """
    Actualiza los contadores de viviendas con más de dos ocupantes y sin baño.

    Params:
        aglomerados (dict): Diccionario que cuenta para cada aglomerado las viviendas con más de dos ocupantes y sin baño.
        AGLOMERADO (str): Cadena que representa el código del aglomerado donde se ubica el hogar encuestado.
        IV8 (str): Cadena que representa si la vivienda tiene o no tiene baño.
        IX_TOT (int): Cantidad de habitantantes en el hogar encuestado.
        PONDERA (int): Ponderación de los hogares encuestados que se suma a los contadores.
    """
    if AGLOMERADO not in aglomerados:
        aglomerados[AGLOMERADO] = {'cant': 0}

    # si no tiene baño y tiene mas de 2 habitantes lo agrego
    if IV8 == '2' and IX_TOT > 2:
        aglomerados[AGLOMERADO]['cant'] += PONDERA
        
# Declarar función que imprime
def imprimir_aglomerados_ocupados_sin_banio(aglomerados):
	"""
	Imprime el aglomerado con mayor cantidad de viviendas con más de dos ocupantes y sin baño y su cantidad, utilizando los contadores acumulados.

	Params:
		aglomerados (dict): Diccionario que cuenta para cada aglomerado las viviendas con más de dos ocupantes y sin baño.
	"""
	# buscamos el aglomerado con mayor cantidad
	maxAglomerado = max(aglomerados,key=lambda k: aglomerados[k]['cant'])
	maxCant = aglomerados[maxAglomerado]['cant']

	# busco el nombre del aglomerado, si no se encuentra devuelvo el codigo 
	nombre = AGLOMERADO_STR.get(int(maxAglomerado), f"Aglomerado {maxAglomerado}")

	print(f"aglomerado con mayor cantidad de viviendas con más de dos ocupantes y sin baño: {nombre}, cantidad: {maxCant}")    

# Ejercicio 7 Parte B

Informar para cada aglomerado el porcentaje de personas que hayan cursado al
menos en nivel universitario o superior.

In [10]:
# Declarar función contadora
def contar_personas_uni_o_sup(aglomerados, AGLOMERADO, PONDERA,NIVEL_ED_str):
	"""
	Actualiza los contadores de personas que hayan cursado al menos en nivel universitario o superior por cada aglomerado.

	Params:
		aglomerados (dict): Diccionario que cuenta para cada aglomerado la poblacion total y la cantidad de personas que hayan cursado al menos en nivel universitario o superior.
		AGLOMERADO (int): Numero que representa el código del aglomerado donde se ubica el encuestado.
		PONDERA (int): Ponderación de los encuestados que se suma a los contadores.
		NIVEL_ED_str (str): Nivel educativo alcanzado por el encuestado.
	"""
	# Si el aglomerado no fue registrado aún, lo inicializamos
	if AGLOMERADO not in aglomerados:
		aglomerados[AGLOMERADO] = {"cant_perso": 0, "cant_universitarias": 0}

	# Sumamos la población ponderada al total del aglomerado
	aglomerados[AGLOMERADO]["cant_perso"] += PONDERA

	# Si el nivel educativo es universitario o superior, lo sumamos
	if NIVEL_ED_str == "Superior o universitario":
		aglomerados[AGLOMERADO]["cant_universitarias"] += PONDERA


# Declarar función que imprime
def imprimir_personas_uni_o_sup(aglomerados):
	"""
	Imprime para cada aglomerado el porcentaje de personas que hayan cursado al menos en nivel universitario o superior, utilizando los contadores acumulados.

	Params:
		aglomerados (dict): Diccionario que cuenta para cada aglomerado la poblacion total y la cantidad de personas que hayan cursado al menos en nivel universitario o superior.
	"""
	print("Porcentaje de personas que hayan cursado al menos en nivel universitario o superior por aglomerado: \n")
	# Calculamos el porcentaje y lo imprimimos
	for cod_aglomerado in aglomerados:
		porcentaje= (aglomerados[cod_aglomerado]["cant_universitarias"] * 100) / aglomerados[cod_aglomerado]["cant_perso"]
		print (f"{AGLOMERADO_STR.get(cod_aglomerado) or cod_aglomerado} : {porcentaje}%")

# Ejercicio 8

ordenar las regiones de forma descendente segun el procentaje de 
inquilinos en cada una

REGION = 
    01 = Gran Buenos Aires
    40 = Noroeste
    41 = Noreste
    42 = Cuyo
    43 = Pampeana
    44 = Patagonia         
PONDERA 

In [11]:
# Definimos el diccionario con los códigos de región y sus nombres
REGION_STR = {
    '1': 'Gran Buenos Aires',
    '40': 'Noroeste',
    '41': 'Noreste',
    '42': 'Cuyo',
    '43': 'Pampeana',
    '44': 'Patagonia'
}

# Declarar función contadora
def contar_inquilinos_por_region(contador_regiones,REGION,PONDERA,IX_Tot,II7):
	"""
	Actualiza los contadores de cantidad total de personas e inquilinos por cada región.

	Params:
		contador_regiones (dict): Diccionario que cuenta para cada región la poblacion total y la cantidad de inquilinos.
		REGION (str): Numero que representa el código de la región donde se ubica el hogar encuestado.
		PONDERA (int): Ponderación de los hogares encuestados que se suma a los contadores.
		IX_Tot (int): Cantidad de habitantes del hogar encuestado.
		II7 (str): Cadena que representa el régimen de tenencia del hogar.
	"""
	if REGION not in contador_regiones:
		contador_regiones[REGION] = {'total':0,'inquilinos':0}

	# Sumo a los totales de la región
	contador_regiones[REGION]['total'] += PONDERA * IX_Tot
	# Sumo los inquilinos
	if II7 == '3':
		contador_regiones[REGION]['inquilinos'] += PONDERA

# Declarar función que imprime 
def imprimir_inquilinos_por_region_ordenado(contador_regiones):
	"""
	Imprime para cada región el porcentaje de personas inquilinas.

	Params:
		contador_regiones (dict): Diccionario que cuenta para cada región la poblacion total y la cantidad de inquilinos, utilizando los contadores acumulados.
	"""
	# calcula el porcentaje de inquilinos
	for codigo_region, data in contador_regiones.items():
		total = data['total']
		inquilinos = data['inquilinos']
		if total > 0:
			porcentaje_inquilinos = (inquilinos / total) * 100
			contador_regiones[codigo_region]['porcentaje_inquilinos'] = porcentaje_inquilinos
		else:
			contador_regiones[codigo_region]['porcentaje_inquilinos'] = 0

	# Ordena las regiones por porcentaje de inquilinos de forma descendente
	regiones_ordenadas = sorted(contador_regiones.items(), key=lambda x: x[1]['porcentaje_inquilinos'], reverse=True)

	# Imprime el resultado
	print(HEADER)
	print("Porcentaje de inquilinos por región (de mayor a menor):\n")
	for codigo_region, data in regiones_ordenadas:
		# Obtener el nombre de la región usando la constante REGION_STR
		nombre_region = REGION_STR.get(codigo_region, f"Región desconocida ({codigo_region})")
		print(f"Región: {nombre_region}, Porcentaje de inquilinos: {data['porcentaje_inquilinos']:.2f}%")
	print(HEADER)

# Ejercicio 9 - Parte B

Se debe pedir al usuario que seleccione un aglomerado y a partir de la información contenida
retornar una tabla que contenga la cantidad de personas mayores de edad según su
nivel de estudios alcanzados.



In [12]:
# Declarar función contadora
def contar_estudios_en_aglomerado(contador_estudios,aglomerado_usuario,AGLOMERADO,ANO4,TRIMESTRE,CH06,NIVEL_ED_str,PONDERA):
	"""
	Actualiza los contadores de personas mayores de edad segun su nivel de estudios.

	Params:
		contador_estudios (defaultdict): Contador que almacena para cada año y trimestre la cantidad de personas según su nivel de estudios.
		aglomerado_usuario (str): Numero que representa el código del aglomerado elegido por el usuario para contar.
		AGLOMERADO (str): Numero que representa el código del aglomerado donde se ubica el encuestado.
		ANO4 (str): Año de la encuesta.
		TRIMESTRE (str): Trimestre de la encuesta.
		CH06 (int): Edad del encuestado.
		NIVEL_ED_str (str): Nivel educativo alcanzado por el encuestado.
		PONDERA (int): Ponderación de los encuestados que se suma a los contadores.
	"""
	if AGLOMERADO == aglomerado_usuario:
		if CH06 >= 18:
			if NIVEL_ED_str == 'Primario incompleto':
				contador_estudios[ANO4][TRIMESTRE]['Primario incompleto'] += PONDERA
			elif NIVEL_ED_str == 'Primario completo':
				contador_estudios[ANO4][TRIMESTRE]['Primario completo'] += PONDERA
			elif NIVEL_ED_str == 'Secundario incompleto':
				contador_estudios[ANO4][TRIMESTRE]['Secundario incompleto'] += PONDERA
			elif NIVEL_ED_str == 'Secundario completo':
				contador_estudios[ANO4][TRIMESTRE]['Secundario completo'] += PONDERA
			elif NIVEL_ED_str  == "Superior o universitario":
				contador_estudios[ANO4][TRIMESTRE]['Superior o universitario'] += PONDERA

# Declarar función que imprime 
def imprimir_estudios_en_aglomerado(contador_estudios,aglomerado_usuario):
	"""
	Imprime para cada año y trimestre la cantidad de personas según su nivel de estudios dentro del aglomerado elegido por el usuario, 
	utilizando los contadores acumulados.

	Params:
		contador_estudios (defaultdict): Contador que almacena para cada año y trimestre la cantidad de personas según su nivel de estudios.
		aglomerado_usuario (str): Numero que representa el código del aglomerado elegido por el usuario para contar.
	"""
	print(HEADER)
	print(f"Imprimiendo personas mayores de edad segun su nivel de estudios en el aglomerado {aglomerado_usuario}.")

	# Imprimir encabezado
	print(f"\nNombre de Aglomerado: {AGLOMERADO_STR[int(aglomerado)]}") # Ya se comprobó antes que fuera una clave de AGLOMERADO_STR por lo que accedo sin get
	print(f"{'Año':<6}{'Trimestre':<12}{'Prim.Inc.':<12}{'Prim.Comp.':<12}"
		f"{'Sec.Inc.':<12}{'Sec.Comp.':<12}{'Sup.Univ.':<12}")
	print(SEPARADOR)

	# Imprimir tabla
	for año in sorted(contador_estudios, key=int):
		for trim in sorted(contador_estudios[año], key=int):
			fila = contador_estudios[año][trim]
			print(f"{año:<6}{trim:<12}{fila['Primario incompleto']:<12}{fila['Primario completo']:<12}" #utilizo especificadores de formato para darle el formato de tabla requerido
				f"{fila['Secundario incompleto']:<12}{fila['Secundario completo']:<12}"
				f"{fila['Superior o universitario']:<12}")
	print(HEADER)


# Ejercicio 10 Parte B

Se debe pedir al usuario que seleccione dos aglomerados y a partir de la información
contenida retornar una tabla que contenga el porcentaje de personas mayores de
edad con secundario incompleto.


In [13]:
# Declarar función contadora
def contar_adultos_sec_inc(contador_adultos_sec_inc, agglomerate_a, agglomerate_b, AGLOMERADO, PONDERA, ANO4, TRIMESTRE, CH06, NIVEL_ED_str):
	"""
	Actualiza los contadores para cada año y trimestre, registrando la cantidad de adultos con secundario incompleto en los aglomerados elegidos por el usuario.

	Params:
		contador_adultos_sec_inc (defaultdict): Contador que almacena para cada año y trimestre la cantidad de adultos con el secundario incompleto en los aglomerados elegidos por el usuario.
		agglomerate_a (str): Primer aglomerado elegido por el usuario.
		agglomerate_b (str): Segundo aglomerado elegido por el usuario.
		AGLOMERADO (str): Numero que representa el código del aglomerado donde se ubica el encuestado.
		PONDERA (int): Ponderación de los encuestados que se suma a los contadores.
		ANO4 (str): Año de la encuesta.
		TRIMESTRE (str): Trimestre de la encuesta.
		CH06 (int): Edad del encuestado.
		NIVEL_ED_str (str): Nivel educativo alcanzado por el encuestado.
	"""
	if AGLOMERADO == agglomerate_a or AGLOMERADO == agglomerate_b:
		contador_adultos_sec_inc[ANO4][TRIMESTRE][AGLOMERADO]['total'] += PONDERA

		# Si es mayor de edad con secundario incompleto
		if CH06 >= 18 and NIVEL_ED_str == 'Secundario incompleto':
			contador_adultos_sec_inc[ANO4][TRIMESTRE][AGLOMERADO]['incomplete'] += PONDERA
               
def imprimir_adultos_sec_inc(contador_adultos_sec_inc, agglomerate_a, agglomerate_b):
	"""
	Imprime una tabla que contiene para cada año y trimestre el porcentaje de personas mayores de edad con el secundario incompleto 
    dentro de los aglomerados elegidos por el usuario, utilizando los contadores acumulados.

	Params:
		contador_adultos_sec_inc (defaultdict): Contador que almacena para cada año y trimestre la cantidad de adultos con el secundario incompleto en los aglomerados elegidos por el usuario.
		agglomerate_a (str): Primer aglomerado elegido por el usuario.
		agglomerate_b (str): Segundo aglomerado elegido por el usuario.
	"""
	agglomerate_a_str = AGLOMERADO_STR[int(agglomerate_a)]
	agglomerate_b_str = AGLOMERADO_STR[int(agglomerate_b)]
      
	print(f"Imprimiendo porcentaje de personas mayores de edad con el secundario incompleto dentro de los aglomerados {agglomerate_a_str} y {agglomerate_b_str}\n")
      
	print(f'{SEPARADOR}\nAño  | Trimestre | Aglomerado {agglomerate_a_str} | Aglomerado {agglomerate_b_str}\n{SEPARADOR}')

	# Calcular cantidad de espacios
	spaces_a = 13 + len(agglomerate_a_str)
	spaces_b = 13 + len(agglomerate_b_str)

	# Recorrer e imprimir cada trimestre
	for year in sorted(contador_adultos_sec_inc.keys(),key=int):
		for quarter in sorted(contador_adultos_sec_inc[year],key=int):
			stats_a = contador_adultos_sec_inc[year][quarter][agglomerate_a]
			stats_b = contador_adultos_sec_inc[year][quarter][agglomerate_b]
			# Calcular porcentajes
			pct_a  = (stats_a['incomplete'] / stats_a['total'] * 100) if stats_a['total'] else 0
			pct_b = (stats_b['incomplete'] / stats_b['total'] * 100) if stats_b['total'] else 0
			# Imprimir fila
			print(f"{year:<5}|{quarter:^11}|{(f'{pct_a :.2f}%'):^{spaces_a}}|{(f'{pct_b :.2f}%'):^{spaces_b}}\n{SEPARADOR}")

# Ejercicio 11 Parte B
Pedir al usuario que seleccione un año, y busque en el último trimestre almacenado del mencionado año, el aglomerado con mayor porcentaje de viviendas de "Material precario" y el aglomerado con menor porcentaje de viviendas de "Material precario".


In [14]:
# Solicita al usuario que ingrese el año
year = int(input("Seleccione un año para analizar el material de techumbre: "))

# Hace el análisis usando la función importada
resultado = analizar_material_precario(datos_hogares, year)

# Mostrar resultados
if 'error' in resultado:
    print(resultado['error'])
else:
    print(f"\nResultados para el año {resultado['año']}, trimestre {resultado['trimestre']}:")
    print(f"Aglomerado con mayor porcentaje de viviendas de 'Material precario': {resultado['max_aglomerado_nombre']} con {resultado['max_porcentaje']:.2f}%")
    print(f"Aglomerado con menor porcentaje de viviendas de 'Material precario': {resultado['min_aglomerado_nombre']} con {resultado['min_porcentaje']:.2f}%")
    
    # Mostrar todos los aglomerados ordenados por porcentaje
    print("\nPorcentaje de viviendas con material precario por aglomerado:")
    for nombre, porcentaje in resultado['porcentajes_ordenados']:
        print(f"{nombre}: {porcentaje:.2f}%")


Resultados para el año 2024, trimestre 2:
Aglomerado con mayor porcentaje de viviendas de 'Material precario': Gran San Juan con 12.38%
Aglomerado con menor porcentaje de viviendas de 'Material precario': Jujuy - Palpalá con 0.00%

Porcentaje de viviendas con material precario por aglomerado:
Gran San Juan: 12.38%
Gran La Plata: 7.59%
Mar del Plata: 5.56%
San Nicolás – Villa Constitución: 5.47%
Gran Catamarca: 2.95%
Concordia: 2.57%
Río Gallegos: 2.46%
Comodoro Rivadavia - Rada Tilly: 2.31%
Neuquén – Plottier: 1.98%
Formosa: 1.48%
Gran Resistencia: 1.45%
Santa Rosa – Toay: 1.37%
Gran Rosario: 1.30%
La Rioja: 1.25%
Gran Mendoza: 1.21%
Gran Tucumán - Tafí Viejo: 1.17%
Corrientes: 1.17%
Partidos del GBA: 1.03%
Gran Salta: 0.99%
Gran Córdoba: 0.90%
Gran San Luis: 0.59%
Bahía Blanca - Cerri: 0.37%
Ciudad Autónoma de Buenos Aires: 0.29%
Ushuaia - Río Grande: 0.25%
Posadas: 0.24%
Gran Santa Fé: 0.22%
Río Cuarto: 0.19%
Santiago del Estero - La Banda: 0.14%
Jujuy - Palpalá: 0.00%
Gran Paraná: 

# Ejercicio 12 Parte B

A partir de la información del último trimestre almacenado en el sistema se debe
retornar para cada aglomerado el porcentaje de jubilados que vivan en una vivienda
con CONDICION_DE_HABITABILIDAD insuficiente.

In [15]:
''' 
Diccionario que contendra una clave que sera un tupla con CODUSU y NRO_HOGAR, el valor son los datos del hogar, esta estructura
    sera util para poder recuperar el hogar de una persona rapidamente.
'''
indice_hogares = {}

# Extraigo solo la informacion necesaria de los individos para poder procesarlos
datos_individuos_CAT_INAC = extraer(INDIVIDUAL_DATA_OUT_PATH, ['CAT_INAC', 'AGLOMERADO', 'PONDERA', 'CODUSU', 'NRO_HOGAR'])

def cargar_datos_aglomera_jubilado_insuficiente():
    ''' 
        Retorno un diccionario donde la clave es un aglomerado y el valor son la cantidad de jubilados en ese aglomerado y la cantidad de jubilados cuyo hogar esta en condicion de insuficiencia
    '''
    datos_aglomerado_jubilado_insuficiente = {}

    for habitante in datos_individuos_CAT_INAC:
        if (int(habitante['AGLOMERADO']) not in datos_aglomerado_jubilado_insuficiente):
            datos_aglomerado_jubilado_insuficiente[int(habitante["AGLOMERADO"])] = {"cant_jubilados": 0, "cant_jubilados_insuficiente": 0}
        if (habitante['CAT_INAC'] == '1'):
            datos_aglomerado_jubilado_insuficiente[int(habitante["AGLOMERADO"])]["cant_jubilados"] += int(habitante["PONDERA"])
            # Busco el hogar al que pertenece el habitante
            hogar = indice_hogares.get((habitante["CODUSU"], habitante["NRO_HOGAR"]))
            # Si ademas el hogar esta en condicion insuficiente, lo contabilizo
            if (hogar == "insuficiente"):
                datos_aglomerado_jubilado_insuficiente[int(habitante["AGLOMERADO"])]["cant_jubilados_insuficiente"] += int(habitante["PONDERA"])

    return datos_aglomerado_jubilado_insuficiente

def porcentaje_jubilados_insuficiente_por_aglomerado (datos_aglomerado):
    '''
        Recibe un diccionario donde cada aglomerado guarda la cantidad de jubilados que posee y la cantidad de jubilados que viven en hogar con condicion de habitabilidad insuficiente.

        return: retorna un diccionario donde para cada aglomerado se tiene el porcentaje de jubilado cuyo hogar tiene condicion de habitabilidad insuficiente.
    '''
    porcentaje_aglomerado = {}
    for cod_aglomerado in datos_aglomerado:
        porcentaje_aglomerado[AGLOMERADO_STR [cod_aglomerado]] = (
            datos_aglomerado[cod_aglomerado]["cant_jubilados_insuficiente"] * 100)/datos_aglomerado[cod_aglomerado]["cant_jubilados"]
    return porcentaje_aglomerado


# Ejercicio 13 Parte B

Solicitar un año al usuario y a partir de la información del último trimestre de dicho 
ejercicio informar la cantidad de personas que hayan cursado nivel universitario o 
superior y que vivan en una vivienda con CONDICION_DE_HABITABILIDAD 
insuficiente 



In [16]:
indiviudos_insuficiente_por_hogar = {}

try: 
    trims = get_quarters()
except FileNotFoundError:
    print(f"No existe la carpeta 'hogares' o 'individuos'.")
except PermissionError:
    print(f"No tienes permiso para acceder a la carpeta 'hogares' o 'individuos'.")

def cargar_indiviudos_insuficiente_por_hogar (CODUSU, NRO_HOGAR, PONDERA):
    if (not ((CODUSU, NRO_HOGAR) in indiviudos_insuficiente_por_hogar)):
        indiviudos_insuficiente_por_hogar [(CODUSU, NRO_HOGAR)] = 0
    indiviudos_insuficiente_por_hogar [(CODUSU, NRO_HOGAR)] = indiviudos_insuficiente_por_hogar [(CODUSU, NRO_HOGAR)] + PONDERA



# Solicitar entrada del usuario cuando se requiera

In [17]:
# Entrada del ejercicio 2
while True:
    anio_usuario = input("Ingrese el año para contar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior: ")
    if anio_usuario.isdigit():
        break
    else:
        print("Error: El año ingresado no es válido. Debe ser un número")

while True:
    trimestre_usuario = input("Ingrese el trimestre para contar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior: ")
    if trimestre_usuario.isdigit() and 1 <= int(trimestre_usuario) <= 4:
        break
    else:
        print("Error: El trimestre ingresado no es válido. Debe ser un número entre 1 y 4.")
        
# Entrada del ejercicio 9
while True:
    aglomerado_usuario = input("Ingrese el número del aglomerado para contar personas mayores de edad según su nivel de estudios: ")
    if not aglomerado_usuario.isdigit():
        print("Debe ingresar un número válido.")
    elif int(aglomerado_usuario) not in AGLOMERADO_STR:
        print("Debe ingresar un número de aglomerado existente.")
    else:
        break 
    
# Entrada del ejercicio 10
while True:
    agglomerate_a = input('Ingrese el primer código de aglomerado para calcular el porcentaje de personas mayores de edad con el secundario incompleto: ').strip()
    agglomerate_b = input('Ingrese el segundo código de aglomerado para calcular el porcentaje de personas mayores de edad con el secundario incompleto: ').strip()

    # Validar que sean numéricos
    if not (agglomerate_a.isnumeric() and agglomerate_b.isnumeric()):
        print('Ambos códigos deben ser números.')
    else:
        # Validar existencia en el diccionario
        if int(agglomerate_a) not in AGLOMERADO_STR:
            print(f'El código {agglomerate_a} no se encuentra en la lista de aglomerados.')
        elif int(agglomerate_b) not in AGLOMERADO_STR:
            print(f'El código {agglomerate_b} no se encuentra en la lista de aglomerados.')
        else:
            break

# Ejercicio 13
años_disponibles = {y for (y, _) in trims}
while True:
    anio_user = input("Ingrese un año: ")
    if not anio_user.isnumeric():
        print("ERROR: debe ingresar un año numérico")
    else:
        anio_user = int(anio_user)
        if anio_user not in años_disponibles:
            print("El año ingresado no se encuentra")
        else:
            trimestres_del_año = [t for (y, t) in trims if y == anio_user]
            trimestre_mas_actual_de_un_year = max(trimestres_del_año)
            break


# Declarar contadores para cada ejercicio

In [18]:
# Ejercicio 1
contador_analfabetos = defaultdict(lambda: {'trim': 0, 'target': 0, 'total': 0})

# Ejercicio 2
contador_universitarios_no_argentinos = {
    "total_no_nacidos_arg": 0,
    "universitarios_no_nacidos_arg": 0
}

# Ejercicio 3
anos_desocupacion = {}

# Ejercicio 5
aglomerados_propietarios = {}

# Ejercicio 6
aglomerados_sin_banio = {}

# Ejercicio 7
datos_aglomerado = {}

# Ejercicio 8
contador_regiones = {}

# Ejercicio 9
contador_estudios = defaultdict(lambda: defaultdict(lambda: {
	'Primario incompleto': 0,
	'Primario completo': 0,
	'Secundario incompleto': 0,
	'Secundario completo': 0,
	'Superior o universitario': 0
}))

# Ejercicio 10
contador_adultos_sec_inc = defaultdict(
    lambda: defaultdict(
        lambda: {
            agglomerate_a: {'total': 0, 'incomplete': 0},
            agglomerate_b: {'total': 0, 'incomplete': 0}
        }
    )
)

# Ejercicio 13
cantidad_personas_insuficiente_universitarios = 0

# Procesar el archivo

In [19]:
for individuo in datos_individuos:
    ano4 = individuo["ANO4"]
    trimestre = individuo["TRIMESTRE"]
    pondera = int(individuo["PONDERA"])
    NIVEL_ED_str = individuo["NIVEL_ED_str"]
    aglomerado = individuo["AGLOMERADO"]
    ch06 = int(individuo["CH06"])
    codusu = individuo["CODUSU"]
    nro_hogar = int(individuo["NRO_HOGAR"])
    # Ejercicio 1
    contar_analfabetos(contador_analfabetos, ano4, int(trimestre), pondera, ch06, individuo["CH09"])
    # Ejercicio 2
    contar_universitarios_no_argentinos(contador_universitarios_no_argentinos, anio_usuario, trimestre_usuario, ano4, trimestre, pondera, individuo["CH15"], NIVEL_ED_str)
    # Ejercicio 3
    contar_desocupacion(anos_desocupacion, individuo["ESTADO"], ano4, pondera)
    # Ejercicio 7
    contar_personas_uni_o_sup(datos_aglomerado, int(aglomerado), pondera, NIVEL_ED_str)
    # Ejercicio 9
    contar_estudios_en_aglomerado(contador_estudios,aglomerado_usuario,aglomerado,ano4,trimestre,ch06,NIVEL_ED_str,pondera)
    # Ejercicio 10
    contar_adultos_sec_inc(contador_adultos_sec_inc,agglomerate_a,agglomerate_b,aglomerado,pondera,ano4,trimestre,ch06,NIVEL_ED_str)
    # Ejercicio 13
    if ((NIVEL_ED_str == "Superior o universitario") and (trimestre_mas_actual_de_un_year == (int(ano4), int(trimestre)))) :
        cargar_indiviudos_insuficiente_por_hogar (codusu, nro_hogar, pondera)


print(f"{'INDIVIDUOS':*^90}")
# Ejercicio 1
imprimir_analfabetos(contador_analfabetos)
# Ejercicio 2
imprimir_universitarios_no_argentinos(contador_universitarios_no_argentinos)
# Ejercicio 3
imprimir_desocupacion(anos_desocupacion)
# Ejercicio 7
imprimir_personas_uni_o_sup(datos_aglomerado)
# Ejercicio 9
imprimir_estudios_en_aglomerado(contador_estudios,aglomerado_usuario)
# Ejercicio 10
imprimir_adultos_sec_inc(contador_adultos_sec_inc,agglomerate_a,agglomerate_b)

for hogar in datos_hogares:
    ano4 = hogar["ANO4"]
    trimestre = hogar["TRIMESTRE"]
    pondera = int(hogar["PONDERA"])
    aglomerado = int(hogar["AGLOMERADO"])
    ix_tot = int(hogar["IX_TOT"])
    ii7 = hogar["II7"]
    # Ejercicio 5
    contar_viviendas_propietarias(aglomerados_propietarios, aglomerado, ii7,pondera)
    # Ejercicio 6
    contar_aglomerados_ocupados_sin_banio(aglomerados_sin_banio, aglomerado,hogar["IV8"], ix_tot,pondera)
    # Ejercicio 8
    contar_inquilinos_por_region(contador_regiones, hogar["REGION"], pondera, ix_tot,ii7)
    # Ejercicio 12
    indice_hogares [(hogar['CODUSU']), hogar['NRO_HOGAR']] = hogar["CONDICION_DE_HABITABILIDAD"]
    # Ejercicio 13
    if ((trimestre_mas_actual_de_un_year == (int(ano4), int(trimestre))) and (hogar["CONDICION_DE_HABITABILIDAD"] == "Insuficiente")):
        cantidad_personas_insuficiente_universitarios = cantidad_personas_insuficiente_universitarios + indiviudos_insuficiente_por_hogar [(hogar["CODUSU"], hogar["NRO_HOGAR"])]


print(f"{'HOGARES':*^90}")
# Ejercicio 5
imprimir_viviendas_propietarias(aglomerados_propietarios)
# Ejercicio 6
imprimir_aglomerados_ocupados_sin_banio(aglomerados_sin_banio)
# Ejercicio 8
imprimir_inquilinos_por_region_ordenado(contador_regiones)

print ('Los 5 aglomerados con mayor porcentaje de hogares con dos o mas habitantes con estudios universitarios o superiores son : ')
print (ranking)

print ('-'*90)

# Ejercicio 12
jubilados_datos_aglomerado = cargar_datos_aglomera_jubilado_insuficiente()
jubilados_insuficiente = porcentaje_jubilados_insuficiente_por_aglomerado (jubilados_datos_aglomerado)

for elem in jubilados_insuficiente:
    print (f'En {elem} el {jubilados_insuficiente[elem]}% de los jubilados vive en un hogar con condicion de habitabilidad INSUFICIENTE')

print ('-'*90)
# Ejercicio 13
print(f"cantidad de personas que hayan cursado nivel universitario o sup y que vivan con CONDICION_DE_HABITABILIDAD insuficiente: {cantidad_personas_insuficiente_universitarios}")

****************************************INDIVIDUOS****************************************
------------------------------------------------------------------------------------------
Año: 2024, trimestre: 2
El porcentaje de personas mayores a 6 años capaces de leer y escribir en 2024 fue 99.16%
El porcentaje de personas mayores a 6 años incapaces de leer y escribir en 2024 fue 0.84%
------------------------------------------------------------------------------------------
El porcentaje de personas no nacidas en Argentina con nivel universitario o superior en el año 2024, trimestre 1: 0.00%
------------------------------------------------------------------------------------------
Año: 2024, Desocupados: 1088226

El año con menor desocupación es 2024 con una total de 1088226 de desocupados.
------------------------------------------------------------------------------------------
Porcentaje de personas que hayan cursado al menos en nivel universitario o superior por aglomerado: 

Ciudad A